In [32]:
%run portugal_functions.py

In [27]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from ctgan import CTGAN
from mapie.metrics import (
    classification_coverage_score,
    classification_mean_width_score
)
from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.calibration import CalibratedClassifierCV
from typing import Tuple, Dict, Union, List
from mapie.classification import MapieClassifier
from mapie.mondrian import MondrianCP
from mapie.metrics import (
    classification_coverage_score,
    classification_mean_width_score
)
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_excel("comparative_data/data.xlsx")

In [51]:
df1 = df.copy()
df1["source"] = "real"
df1 = df1.rename(columns={'Instrument':'email'})

nb_nan_par_ligne = df1.isna().sum(axis=1)
df1 = df1[ nb_nan_par_ligne != 46 ]

In [52]:
item_cols = [c for c in df1.columns if c.startswith("Item")]

In [6]:
n_clusters_simple = 5

X = df1.copy()
X = X.drop(columns=['email', 'dropout', 'source'] + item_cols).fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=n_clusters_simple, random_state=42)
clusters = kmeans.fit_predict(X_scaled)
df2 = df1.copy()
df2['cluster'] = clusters
print(pd.Series(clusters).value_counts().sort_index())

0    122
1    246
2    117
3    143
4    229
Name: count, dtype: int64


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


In [7]:
# SMOTE
n_by_cluster_min = 100
k_neighbors = 20

df4 = oversample_minority_clusters(
    df2,
    item_cols,
    n_by_cluster_min=n_by_cluster_min,
    k_neighbors=k_neighbors,
    cluster_col='cluster',
    drop_cols=['email', 'source']
)

In [ ]:
# GAN

# `epochs` : nombre de passes sur les données
# `batch_size` : taille de chaque lot
# `generator_dim` et `discriminator_dim` définissent les architectures
epochs = 300
batch_size=100
n_samples = 200

ctgan1 = CTGAN(
    epochs=epochs,
    batch_size=batch_size,
    generator_dim=(256, 256),
    discriminator_dim=(256, 256),
    verbose=True
)

# 4. Entraînement
# Le modèle apprendra la distribution de vos données
X = df1.copy()
X = X.drop(columns=['email', 'source']).fillna(0)
ctgan1.fit(X)

synthetic_data = ctgan1.sample(n_samples)
synthetic_data['source'] = 'synth'

df5 = pd.concat([df1, synthetic_data], ignore_index=True)

Gen. (-2.24) | Discrim. (0.44):   2%|▏         | 6/300 [00:15<12:30,  2.55s/it] 


KeyboardInterrupt: 

In [ ]:
tasks = [
    ("avec enrichissement, technique GAN", df5, {})
]

summary_records = []
res_classical_gan = []
for name, dfk, kwargs in tasks:
    # 1) Lancement de la fonction
    df_detail, df_agg, y_cible, tr_clf = run_analysis_port(
        df=dfk.drop(columns=['dropout']),
        y=dfk['dropout'],
        alpha = 0.05,
        do_plot=False,
        **kwargs
    )
    r =  df_detail.groupby(["method", "model", "cluster"]).agg(
            mean_coverage=("coverage", "mean"),
            mean_width=("width", "mean")
        ).reset_index()
    res_classical_gan.append(r[r['cluster'] == -1])

In [ ]:
res_classical_gan[0]

In [ ]:
tasks = [
    ("sans rien", df1, {}),
    ("sans enrichissement", df2, {}),
    ("avec enrichissement, technique SMOTE", df4, {}),
    ("avec enrichissement, technique GAN", df5, {})
]

summary_records = []
res_classical = []
for name, dfk, kwargs in tasks:
    # 1) Lancement de la fonction
    df_detail, df_agg, y_cible, tr_clf = run_analysis_port(
        df=dfk.drop(columns=['dropout']),
        y=dfk['dropout'],
        alpha = 0.05,
        do_plot=False,
        **kwargs
    )
    r =  df_detail.groupby(["method", "model", "cluster","n_projects"]).agg(
            mean_coverage=("coverage", "mean"),
            mean_width=("width", "mean")
        ).reset_index()
    res_classical.append(r[r['cluster'] == -1])

GB: 100%|██████████| 42/42 [00:19<00:00,  2.10it/s]


In [9]:
w1 = 3 #PARAM w

# 1. Sélection des colonnes « Item » (ici on prend les 43 premières si besoin)
item_cols = [c for c in df1.columns if c.startswith("Item")]

# 2. Colonnes statiques (sans les Items ni les métadonnées)
static_cols = [
    c for c in df1.columns
    if c not in item_cols + ["email", "dropout", "source", "cluster"]
]

# 3. DataFrame de base, qu’on ne modifie pas en place
base_df = df1[static_cols].copy()

# 4. Construction cumulative du dictionnaire Xt
Xt = {"t0": base_df.copy()}
cum_df = base_df.copy()
for idx, col in enumerate(item_cols, start=1):
    cum_df = cum_df.copy()
    cum_df[f"t{idx}"] = df1[col]
    Xt[f"t{idx}"] = cum_df

# 5. Construire y pour l’horizon H
H = 1
keys = list(Xt.keys())
y = {}

# cible pour t0
if H < len(keys):
    y["t0"] = Xt[keys[H]].iloc[:, -1].copy().fillna(0)
else:
    y["t0"] = Xt[keys[-1]].iloc[:, -1].copy().fillna(0)

for i, key in enumerate(keys[1:], start=1):
    tgt = i + H
    if tgt < len(keys):
        df_tgt = Xt[keys[tgt]]
    else:
        df_tgt = Xt[keys[-1]]
    y[key] = df_tgt.iloc[:, -1].copy().fillna(0)

# 6. Construire X en ne gardant que les w dernières notes (fenêtre glissante)
X = {}

# on parcourt les mêmes clés que pour y, mais on saute celles où i < w
for i, key in enumerate(keys):
    if i < w1:
        continue
    # on prend les w derniers item_cols correspondant aux notes t_{i-w+1} … t_i
    window_item_cols = item_cols[i-w1 : i]
    X[key] = df1[window_item_cols].copy().fillna(0)

# 7. Conversion en arrays NumPy (alignés sur les mêmes clés)
valid_keys = keys[w1:]  # on commence à t{w}
X_array_hori = [X[k].values for k in valid_keys]
y_array_hori = [y[k].values for k in valid_keys]

X_array = X_array_hori.copy()
y_array = y_array_hori.copy()
U_t = []
# On parcourt i de 1 à len(X_array)-1 (i=0 n'a pas de passé pour entraîner)
for i in tqdm(range(1, len(X_array) - 1), desc="Fenêtres en ligne"):
    # --- 1) Construction du train sur les fenêtres passées ---
    X_train = np.vstack(X_array[:i])      # fenêtres 0..i-1
    y_train = np.concatenate(y_array[:i])
    # --- 2) Entraînement d'un nouveau modèle ---
    model = OneSidedSPCI_LGBM_Offline(alpha=0.1, w=300, random_state=0)
    model.fit(X_train, y_train)
    X_i, y_i = X_array[i], y_array[i]
    # calcul des bornes supérieures U_t pour chaque échantillon de X_i
    U = np.array([
        model.predict_interval(x.reshape(1, -1))[1]
        for x in X_i
    ])
    U_t.append(U)

df6 = df1.copy()

# 1) nombre de colonnes avant insertion
n_cols = df6.shape[1]

# 2) on insère chaque vecteur de U_t avant la nième colonne depuis la fin
#    et on génère le nom "Item_{42-i}_next_grade"
for i, ut in enumerate(reversed(U_t), start=1):
    loc = n_cols - 1 - i
    col_name = f"Item {42 - i}_next_grade"
    df6.insert(loc, col_name, ut) # avec notes SPCI

Fenêtres en ligne:   0%|          | 0/38 [00:00<?, ?it/s]

fit 1 ok
fit 2 ok


Fenêtres en ligne:   3%|▎         | 1/38 [00:07<04:21,  7.07s/it]

fit 1 ok
fit 2 ok


Fenêtres en ligne:   5%|▌         | 2/38 [00:20<06:24, 10.69s/it]

...

Fenêtres en ligne:  95%|█████████▍| 36/38 [1:50:21<12:14, 367.43s/it]

fit 1 ok
fit 2 ok


Fenêtres en ligne:  97%|█████████▋| 37/38 [1:57:10<06:19, 379.98s/it]

fit 1 ok
fit 2 ok


Fenêtres en ligne: 100%|██████████| 38/38 [2:03:21<00:00, 194.78s/it]


In [10]:
n_clusters_spci_next_grade = 8
X = df6.drop(columns=['email', 'dropout', 'source'] + item_cols).fillna(0)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=n_clusters_spci_next_grade, random_state=42)
clusters = kmeans.fit_predict(X_scaled)
df7 = df6.copy()
df7['cluster'] = clusters # avec notes SPCI et cluster
print(pd.Series(clusters).value_counts().sort_index())

0    101
1    184
2     74
3     37
4    157
5     79
6    141
7     84
Name: count, dtype: int64


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


In [35]:
min_cluster_size = 100
dfng = assign_clusters_with_min_size(df6, n_clusters=8, min_cluster_size=min_cluster_size)

Clusters trop petits à réaffecter : [0, 4, 5, 6, 7]
Nouvelles tailles de clusters :
 1    314
2    262
3    281
Name: count, dtype: int64


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


In [11]:
df8 = oversample_minority_clusters(
    df7,
    item_cols,
    n_by_cluster_min=n_by_cluster_min,
    k_neighbors=k_neighbors,
    cluster_col='cluster',
    drop_cols=['email', 'source']
)

In [ ]:
ctgan2 = CTGAN(
    epochs=50,
    batch_size=100,
    generator_dim=(256, 256),
    discriminator_dim=(256, 256),
    verbose=True
)

# 4. Entraînement
# Le modèle apprendra la distribution de vos données
X = df6.copy()
X = X.drop(columns=['email', 'source']).fillna(0)
ctgan2.fit(X)


Gen. (-5.06) | Discrim. (0.00): 100%|██████████| 50/50 [07:22<00:00,  8.85s/it] 


NameError: name 'n_samples' is not defined

In [42]:
n_samples = 200
synthetic_data = ctgan2.sample(n_samples)
synthetic_data['source'] = 'synth'

df9 = pd.concat([df6, synthetic_data], ignore_index=True)

In [47]:
tasks = [
    ("GAN enriched and clustered, with next spci grade", df6, {})
]

res_SPCI_next_grade = []
for name, dfk, kwargs in tasks:
    # 1) Lancement de la fonction
    df_detail, df_agg, y_cible, tr_clf = run_analysis_port(
        df=dfk.drop(columns=['dropout']),
        y=dfk['dropout'],
        alpha = 0.05,
        do_plot=False,
        **kwargs
    )
    r =  df_detail.groupby(["method", "model", "cluster","n_projects"]).agg(
            mean_coverage=("coverage", "mean"),
            mean_width=("width", "mean")
        ).reset_index()
    res_SPCI_next_grade.append(r[r['cluster'] == -1])

GB: 100%|██████████| 42/42 [00:25<00:00,  1.68it/s]


In [49]:
res_SPCI_next_grade[0].groupby(["method", "model", "cluster"]).agg(
            mean_coverage=("mean_coverage", "mean"),
            mean_width=("mean_width", "mean")
        ).reset_index()

,method,model,cluster,mean_coverage,mean_width
0,mondrian,GB,-1,0.928571,1.301495
1,mondrian,LR,-1,0.958749,1.617386
2,mondrian,RF,-1,0.916113,1.393688
3,vanilla,GB,-1,0.928571,1.301495
4,vanilla,LR,-1,0.958749,1.617386
5,vanilla,RF,-1,0.916113,1.393688


In [12]:
target = df1['dropout']
cols = [c for c in df1.columns if c.startswith("Item")]
# print(cols[-3:])
shooting = df1[cols[-1:]].mean(axis=1)

s_opt, err_opt = find_best_threshold(shooting, target)

THRESHOLD = s_opt

In [53]:
w2 = 11
ALPHA2 = 0.05

dfsp = df1.copy()
base_df = dfsp[static_cols].copy()

# 4. Construction cumulative du dictionnaire Xt
Xt = {"t0": base_df.copy()}
cum_df = base_df.copy()
for idx, col in enumerate(item_cols, start=1):
    cum_df = cum_df.copy()
    cum_df[f"t{idx}"] = dfsp[col]
    Xt[f"t{idx}"] = cum_df

# 6. Construire X en ne gardant que les w dernières notes (fenêtre glissante)
X = {}
keys = list(Xt.keys())
# on parcourt les mêmes clés que pour y, mais on saute celles où i < w
for i, key in enumerate(keys):
    if i < w2:
        continue
    # on prend les w derniers item_cols correspondant aux notes t_{i-w+1} … t_i
    window_item_cols = item_cols[i-w1 : i]
    X[key] = dfsp[window_item_cols].copy().fillna(0)

# 7. Conversion en arrays NumPy (alignés sur les mêmes clés)
valid_keys = keys[w2:]  # on commence à t{w}
X_array_hori = [X[k].values for k in valid_keys]
models2sp = []
X_array = X_array_hori.copy()
k = len(X_array)
# Parcours des fenêtres temporelles
for i in tqdm(range(1, k), desc="Fenêtres en ligne"):

    H = len(keys) - i - w2
    y = []

    for j in range(w2, len(keys)):
        if j+H < len(keys):
            y_j = Xt[keys[j+H]].iloc[:, 3]
        else:
            y_j = Xt[keys[-1]].iloc[:, 3]
        y.append(y_j)
    y_array = [s.values for s in y]
    # 1) Construction du train sur les fenêtres passées
    X_train = np.vstack(X_array[:i])      # fenêtres 0..i-1
    y_train = np.concatenate(y_array[:i])
    # 2) Entraînement d'un nouveau modèle
    model = TwoSidedSPCI_RFQuant_Offline(alpha=ALPHA2, w=50, random_state=0)
    model.fit(X_train, y_train)

    # Sauvegarde en mémoire
    models2sp.append(model)

    # 3) Évaluation sur la fenêtre courante i
    X_i, y_i = X_array[i], y_array[i]
    L = np.array([
        model.predict_interval(x.reshape(1, -1))[0]
        for x in X_i
    ])
    U = np.array([
        model.predict_interval(x.reshape(1, -1))[1]
        for x in X_i
    ])

Fenêtres en ligne: 100%|██████████| 31/31 [1:58:10<00:00, 228.72s/it]


In [ ]:
threshold = THRESHOLD
y_all = dfsp['dropout'].astype(int)
y_all_arr = y_all.values
print(
    f" threshold = {threshold:.3f} → {y_all.mean() * 100:.1f}% positives"
)
# 2) Charger tous les modèles dans une liste
loaded_models = models2sp
res2sp = {}
covs2sp, widths2sp = {}, {}

# 3) Parcourir chaque modèle et faire vos prédictions d'intervalles
for idx, model in enumerate(loaded_models, start=1):
    print(f"Modèle rechargé pour fenêtre {idx} -> {model}")
    res2sp["n_projects"] = idx + w2
    # Récupère les nouvelles données pour la fenêtre idx
    # (ici on suppose que X_array_hori[0] était la fenêtre 0, etc.)
    X_new = X_array_hori[idx]

    # Prédiction des intervalles [L, U] pour chaque échantillon de X_new
    intervals = [model.predict_interval(x.reshape(1, -1)) for x in X_new]
    # Séparez les bornes inférieures et supérieures
    L_preds, U_preds = zip(*intervals)

    # Affichage rapide
    print("Exemple de borne inférieure :", np.array(L_preds)[0])
    print("Exemple de borne supérieure :", np.array(U_preds)[0])

    # 4) Application du seuil pour générer y_pred
    y_preds2sp = []
    for L, U in zip(L_preds, U_preds):
        if threshold > U:
            # le seuil est au-dessus de l'intervalle → classe positive uniquement
            y_preds2sp.append([1])
        elif threshold < L:
            # le seuil est en-dessous de l'intervalle → classe négative uniquement
            y_preds2sp.append([0])
        else:
            # le seuil est à l'intérieur de l'intervalle → ambiguïté
            y_preds2sp.append([0,1])

    n_samples = len(y_preds2sp)
    n_classes = 2

    # 1) Créer le tableau booléen
    y_pred_bool = np.zeros((n_samples, n_classes), dtype=bool)
    for i, labels in enumerate(y_preds2sp):
        y_pred_bool[i, labels] = True
    cov_all = classification_coverage_score(y_all_arr, y_pred_bool)
    width_all = classification_mean_width_score(y_pred_bool)
    res2sp["coverage"] = cov_all
    res2sp["width"] = width_all

In [ ]:
dataframes = {
    "simple": df1,
    "+ cluster": df2,
    "+ cluster + SMOTE": df4,
    "+ cluster + GAN": df5,
    "+ SPCI next_grade": df6,
    "+ SPCI next_grade + cluster": df7,
    "+ SPCI next_grade + cluster + SMOTE": df8,
    "SPCI next_grade + cluster + GAN": df9
}

In [55]:
# -----------------------------------------------------------------------------
# Configuration & constants
# -----------------------------------------------------------------------------
RANDOM_STATE: int = 42            # Ensures full reproducibility
ALPHA: float = 0.05               # Target mis-coverage level
W: int = 11                        # Sliding-window size
nan_fill = 0
threshold = 2.0

# -----------------------------------------------------------------------------
# Data preparation
# -----------------------------------------------------------------------------
DF = dfng.copy()
DF.fillna(nan_fill, inplace=True)
DF.reset_index(drop=True, inplace=True)
Y_TARGET = DF['dropout'].astype(int)
item_cols = [c for c in DF.columns if c.startswith("Item")]
item_pref = [f"Item {i}" for i in range(1,43)]
# 2) Définir les “préfixes” comme étant ces noms de colonnes
#    (on parcourt ensuite 1 à len(prefixes) pour ajouter les items un à un)
prefixes = item_pref.copy()
static_cols = [
c for c in DF.columns
if c not in item_cols + ["student_id", "email", "dropout", "source", "cluster"]
]
DF.drop(columns='dropout', inplace=True)
loaded_models = models2sp
# -----------------------------------------------------------------------------
# Base models
# -----------------------------------------------------------------------------
MODELS: Dict[str, object] = {
    "RF": RandomForestClassifier(
        n_estimators=1000,
        min_samples_leaf=2,
        class_weight="balanced",
        n_jobs=-1,
        random_state=RANDOM_STATE,
    ),
    "LR": LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        n_jobs=-1,
        random_state=RANDOM_STATE,
    ),
    "GB": GradientBoostingClassifier(random_state=RANDOM_STATE),
}

# -----------------------------------------------------------------------------
# Conformal prediction evaluation loop
# -----------------------------------------------------------------------------
from tqdm import tqdm

res_fin_port: List[pd.DataFrame] = []

for name, base_clf in MODELS.items():
    covs_MCP, width_MCP = [], []
    covs_SPCI, width_SPCI = [], []
    covs_comb, width_comb = [], []
    covs_union,  width_union  = [], []
    for n in tqdm(range(W, len(item_pref)), desc=name):
        gate_clf = RandomForestClassifier(
            n_estimators=300,
            max_depth=None,
            class_weight="balanced",
            random_state=RANDOM_STATE,
            n_jobs=-1,
        )
        # 1) split train / tmp / test
        idx_tmp, idx_test, y_tmp, y_test, cl_tmp, cl_test = train_test_split(
            DF.index, Y_TARGET, DF["cluster"],
            test_size=0.20,
            stratify=Y_TARGET,
            random_state=RANDOM_STATE,
        )
        idx_tr, idx_cal, y_tr, y_cal, cl_tr, cl_cal = train_test_split(
            idx_tmp, y_tmp, cl_tmp,
            test_size=0.40/0.80,
            stratify=y_tmp,
            random_state=RANDOM_STATE,
        )
        # **nouveau** split en deux pour CP vs gate
        idx_cal_cp, idx_cal_gate, y_cal_cp, y_cal_gate, cl_cal_cp, cl_cal_gate = train_test_split(
            idx_cal, y_cal, cl_cal,
            test_size=0.5,
            stratify=y_cal,
            random_state=RANDOM_STATE,
        )
        y_cal_cp   = np.array(y_cal_cp)
        y_cal_gate = np.array(y_cal_gate)
        y_test     = np.array(y_test)
        # mask des “réels” pour toutes les évaluations
        mask_real = DF.loc[idx_test, "source"] == "real"

        # 2) build features
        X_tr        = build_X_(DF.loc[idx_tr],       item_cols, static_cols, n)
        X_cal_cp    = build_X_(DF.loc[idx_cal_cp],   item_cols, static_cols, n)
        X_cal_gate  = build_X_(DF.loc[idx_cal_gate], item_cols, static_cols, n)
        X_test      = build_X_(DF.loc[idx_test],     item_cols, static_cols, n)
        # 3) train base clf + calibrate for MCP
        clf = clone(base_clf)
        clf.fit(X_tr, y_tr)
        calib = CalibratedClassifierCV(clf, cv="prefit", method="sigmoid") \
                    .fit(X_cal_cp, y_cal_cp)

        base_mapie = MapieClassifier(estimator=calib, method="lac", cv="prefit")
        mond_mapie = MondrianCP(mapie_estimator=base_mapie) \
                        .fit(X_cal_cp, y_cal_cp, partition=cl_cal_cp)

        # ---- MCP on TEST ----
        _, yps_van_test = mond_mapie.predict(X_test, alpha=ALPHA, partition=cl_test)
        pset_van_test = yps_van_test[:, :, 0]
        cov_van = classification_coverage_score(y_test[mask_real], pset_van_test[mask_real])
        wid_van = classification_mean_width_score(pset_van_test[mask_real])
        covs_MCP.append(cov_van)
        width_MCP.append(wid_van)
        print("MCP", cov_van, wid_van)
        # ---- SPCI on TEST ----
        model_spci = loaded_models[n - W]
        pos_test   = DF.index.get_indexer(idx_test)
        X_spci_test = X_array_hori[n - W + 1][pos_test]
        intervals = [model_spci.predict_interval(x.reshape(1, -1))
                     for x in X_spci_test]
        L_preds, U_preds = zip(*intervals)

        y_pred_bool_SPCI = np.zeros((len(intervals), 2), dtype=bool)
        for i, (L, U) in enumerate(zip(L_preds, U_preds)):
            if threshold > U:
                y_pred_bool_SPCI[i, 1] = True
            elif threshold < L:
                y_pred_bool_SPCI[i, 0] = True
            else:
                y_pred_bool_SPCI[i, :] = True

        cov_spci = classification_coverage_score(y_test[mask_real],
                                                y_pred_bool_SPCI[mask_real])
        wid_spci = classification_mean_width_score(y_pred_bool_SPCI[mask_real])
        covs_SPCI.append(cov_spci)
        width_SPCI.append(wid_spci)
        print("SPCI", cov_spci, wid_spci)
        ##UNION 
        y_pred_bool_MCP = pset_van_test.astype(bool)
        y_bool_union = y_pred_bool_MCP | y_pred_bool_SPCI
        cov_union = classification_coverage_score(
            y_test[mask_real],
            y_bool_union[mask_real]
        )
        wid_union = classification_mean_width_score(
            y_bool_union[mask_real]
        )
        covs_union.append(cov_union)
        width_union.append(wid_union)
        print("UNION :", cov_union, wid_union)

        # ---- construire la gate sur CAL_GATE ----
        #  a) MCP predictions sur X_cal_gate
        _, yps_van_gate = mond_mapie.predict(
            X_cal_gate, alpha=ALPHA, partition=cl_cal_gate
        )
        pset_cal_cls = yps_van_gate[:, :, 0]

        #  b) SPCI predictions sur X_cal_gate
        pos_cal_gate  = DF.index.get_indexer(idx_cal_gate)
        X_spci_cal    = X_array_hori[n - W + 1][pos_cal_gate]
        intervals_cal = [model_spci.predict_interval(x.reshape(1, -1))
                         for x in X_spci_cal]
        L_cal, U_cal  = zip(*intervals_cal)

        pset_cal_spc = np.zeros_like(pset_cal_cls, dtype=bool)
        for i, (L, U) in enumerate(zip(L_cal, U_cal)):
            if threshold > U:
                pset_cal_spc[i, 1] = True
            elif threshold < L:
                pset_cal_spc[i, 0] = True
            else:
                pset_cal_spc[i, :] = True

        #  c) préparer méta-features & labels pour la gate
        df_sel_arr = []
        labels_g   = []                      # ← on initialise labels_g

        for i in range(len(idx_cal_gate)):
            feat_vec = X_cal_gate[i]
            w_cls    = pset_cal_cls[i].sum()
            w_spc    = pset_cal_spc[i].sum()
            diff     = w_cls - w_spc
            err_cls  = int(y_cal_gate[i] not in np.where(pset_cal_cls[i])[0])
            err_spc  = int(y_cal_gate[i] not in np.where(pset_cal_spc[i])[0])
            if   err_cls == 0 and err_spc == 1:
                gate_y = 0
            elif err_spc == 0 and err_cls == 1:
                gate_y = 1
            elif err_cls == 0 and err_spc == 0:
                gate_y = 0 if w_cls < w_spc else 1
            else:
                gate_y = 2
            labels_g.append(gate_y)           # ← on stocke le label

            meta_vec = np.concatenate([
                 feat_vec,
                 [w_cls, w_spc, diff, err_cls, err_spc]
            ])
            df_sel_arr.append(meta_vec)

        X_gate_train = np.vstack(df_sel_arr)
        gate_clf.fit(X_gate_train, np.array(labels_g))
        # ---- appliquer la gate sur TEST ----
        meta_test_arr = []
        for i in range(len(idx_test)):
            # feat_vec est un array 1D de taille n_features
            feat_vec = X_test[i]
            w_cls = pset_van_test[i].sum()
            w_spc = y_pred_bool_SPCI[i].sum()
            diff = w_cls - w_spc
            # on concatène feat_vec et les 5 features méta
            meta_vec = np.concatenate([
                feat_vec,
                [w_cls, w_spc, diff, 0, 0]    # err_cls=0, err_spc=0
            ])
            meta_test_arr.append(meta_vec)

        # on empile en matrice (n_test × n_features_meta)
        X_gate_test = np.vstack(meta_test_arr)

        # on prédit le choix de la gate
        choices = gate_clf.predict(X_gate_test)
        pset_final = np.zeros_like(pset_van_test, dtype=bool)
        for i, choice in enumerate(choices):
            if choice == 0:
                pset_final[i] = y_pred_bool_MCP[i]
            elif choice == 1:
                pset_final[i] = y_pred_bool_SPCI[i]
            else:
                pset_final[i] = y_pred_bool_MCP[i] | y_pred_bool_SPCI[i]

        cov_c = classification_coverage_score(y_test[mask_real],
                                             pset_final[mask_real])
        wid_c = classification_mean_width_score(pset_final[mask_real])
        covs_comb.append(cov_c)
        width_comb.append(wid_c)
        print("COMBINED", cov_c, wid_c)
    # on agrège les métriques
    n_vals = list(range(W, W + len(covs_MCP)))
    df_metrics = pd.DataFrame({
        "model":             [name] * len(n_vals),
        "n":                 n_vals,
        "coverage_MCP":      covs_MCP,
        "width_MCP":         width_MCP,
        "coverage_SPCI":     covs_SPCI,
        "width_SPCI":        width_SPCI,
        "coverage_union":    covs_union, 
        "width_union":       width_union,
        "coverage_combined": covs_comb,
        "width_combined":    width_comb,
    })
    res_fin_port.append(df_metrics)


RF:   0%|          | 0/31 [00:00<?, ?it/s]

MCP 0.9069767441860465 1.3837209302325582
SPCI 0.7790697674418605 1.5930232558139534
UNION : 0.9767441860465116 1.8953488372093024


RF:   3%|▎         | 1/31 [00:13<06:36, 13.22s/it]

COMBINED 0.7674418604651163 1.2383720930232558
MCP 0.9069767441860465 1.372093023255814
SPCI 0.9593023255813954 1.7732558139534884
UNION : 0.9941860465116279 1.877906976744186


RF:   6%|▋         | 2/31 [00:47<12:18, 25.45s/it]

...

COMBINED 0.9302325581395349 1.4593023255813953
MCP 0.9418604651162791 1.4534883720930232
SPCI 0.872093023255814 1.75
UNION : 0.9883720930232558 1.9534883720930232


GB:  97%|█████████▋| 30/31 [07:08<00:09,  9.41s/it]

COMBINED 0.8546511627906976 1.3372093023255813
MCP 0.9418604651162791 1.4534883720930232
SPCI 0.5755813953488372 1.0
UNION : 0.9767441860465116 1.6395348837209303


GB: 100%|██████████| 31/31 [07:18<00:00, 14.15s/it]

COMBINED 0.5755813953488372 1.0


In [56]:
res_fin_port[0].iloc[:,1:].mean()

n                    26.000000
coverage_MCP          0.917479
width_MCP             1.388222
coverage_SPCI         0.805514
width_SPCI            1.551388
coverage_union        0.976932
width_union           1.831020
coverage_combined     0.771380
width_combined        1.213053
dtype: float64

In [57]:
res_fin_port[2].iloc[:,1:].mean()

n                    26.000000
coverage_MCP          0.941110
width_MCP             1.451425
coverage_SPCI         0.805514
width_SPCI            1.551388
coverage_union        0.989122
width_union           1.846774
coverage_combined     0.782821
width_combined        1.259752
dtype: float64